In [1]:
import torch
import torch.nn as nn
import torch.utils.data as tud
from tqdm.notebook import tqdm
from pprint import pprint
import matplotlib.pyplot as plt
import numpy as np
import importlib
import sys
sys.path.append("../../")
import autoregressive

In [2]:
# chess

with open("data/chess_book.txt", encoding = "utf-8") as file:
    text = file.read()
print(len(text))
print(text[:1000])

556949
﻿The Project Gutenberg EBook of Chess Strategy, by Edward Lasker

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  You may copy it, give it away or
re-use it under the terms of the Project Gutenberg License included
with this eBook or online at www.gutenberg.org/license


Title: Chess Strategy

Author: Edward Lasker

Translator: J. Du Mont

Release Date: November 11, 2012 [EBook #5614]

Language: English


*** START OF THIS PROJECT GUTENBERG EBOOK CHESS STRATEGY ***




Produced by John Mamoun <mamounjo@umdnj.edu>, Charles
Franks, and the Online Distributed Proofreaders website.








INFORMATION ABOUT THIS E-TEXT EDITION



The following is an e-text of "Chess Strategy," second edition, (1915)
by Edward Lasker, translated by J. Du Mont.

This e-text contains the 167 chess and checkers board game
diagrams appearing in the original book, all in the form of
ASCII line drawings. The following is a key to the diagrams:

For chess

In [3]:
len(text)

556949

In [4]:
vocabulary = set(text)
len(vocabulary)

92

In [5]:
char2i = {c:i for i, c in enumerate(sorted(vocabulary), 3)}
char2i["<PAD>"] = 0
char2i["<START>"] = 1
char2i["<END>"] = 2
i2char = {i:c for i, c in enumerate(sorted(vocabulary), 3)}
i2char[0] = "<PAD>"
i2char[1] = "<START>"
i2char[2] = "<END>"

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [7]:
length = 21
lines = []
for i in range(len(text))[:-length]:
    lines.append(text[i:length + i])
print(len(text))
print(len(lines))
print(lines[:5])
encoded = torch.tensor([[char2i[c] for c in l] for l in tqdm(lines)]).to(device).long()
print(encoded.shape)

556949
556928
['\ufeffThe Project Gutenber', 'The Project Gutenberg', 'he Project Gutenberg ', 'e Project Gutenberg E', ' Project Gutenberg EB']



torch.Size([556928, 21])


In [8]:
importlib.reload(autoregressive)    
net = autoregressive.LSTM(char2i, i2char)
net.to(device)
# using a subsample as "dev" set
performance = net.fit(encoded, encoded[:1000], save_path = "checkpoints/autoregressive_lstm.pt")

Model: Autoregressive LSTM
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Embedding dimension: 8
Hidden units: 64
Layers: 2
Dropout: 0.0
Trainable parameters: 59,159

Epoch | Train                 | Development           | Training time
      | Loss     | Error Rate | Loss     | Error Rate |
---------------------------------------------------------------------
    1 |   2.4601 |     59.684 |   3.3709 |     78.035 |         36.92
    2 |   1.7527 |     46.546 |   3.1164 |     73.575 |         73.44
    3 |   1.5131 |     41.576 |   2.9371 |     72.200 |        109.67
    4 |   1.3859 |     38.736 |   2.8369 |     70.275 |        146.38
    5 |   1.3098 |     36.965 |   2.7619 |     67.345 |        183.49


In [9]:
importlib.reload(autoregressive)    
net = autoregressive.LSTM(char2i, i2char)
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.greedy_search(encoded[:5])

print(log_probabilities)
net.tensor2text(indexes)

Model: Autoregressive LSTM
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Embedding dimension: 8
Hidden units: 64
Layers: 2
Dropout: 0.0
Trainable parameters: 59,159

tensor([-19.5982, -18.1931, -16.9650, -19.1240, -19.1656], device='cuda:0')


['\ufeffThe Project Gutenberg the pawn at the pa',
 'The Project Gutenberg the pawn at the paw',
 'he Project Gutenberg the pawn at the pawn',
 'e Project Gutenberg Eom the pawn at the p',
 ' Project Gutenberg EB           Kt-Kt3\n  ']

In [10]:
importlib.reload(autoregressive)    
net = autoregressive.LSTM(char2i, i2char)
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.sample(encoded[:5])

print(log_probabilities)
net.tensor2text(indexes)

Model: Autoregressive LSTM
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Embedding dimension: 8
Hidden units: 64
Layers: 2
Dropout: 0.0
Trainable parameters: 59,159

tensor([-36.9407, -31.6861, -37.3709, -28.8181, -34.5342], device='cuda:0')


['\ufeffThe Project Gutenbernevile fing, and mov',
 'The Project Gutenbergs there\nevent-game o',
 'he Project Gutenberg weorr to whes\n\n     ',
 'e Project Gutenberg Eefenced with other m',
 ' Project Gutenberg EBare collition, oved ']

In [11]:
importlib.reload(autoregressive)    
net = autoregressive.LSTM(char2i, i2char)
net.load_state_dict(torch.load("checkpoints/autoregressive_lstm.pt"))
net.to(device)
indexes, log_probabilities = net.beam_search(encoded[:5])

print(log_probabilities)
pprint([net.tensor2text(t) for t in indexes])

Model: Autoregressive LSTM
Tokens in the in vocabulary: 95
Tokens in the out vocabulary: 95
Embedding dimension: 8
Hidden units: 64
Layers: 2
Dropout: 0.0
Trainable parameters: 59,159

tensor([[-16.8889, -17.8440, -17.8571, -18.0461, -18.2158],
        [ -6.7699,  -8.4641, -10.2892, -10.7763, -10.7949],
        [-15.4294, -15.6800, -15.9391, -16.1423, -16.2700],
        [-16.0051, -16.0439, -16.2139, -16.4134, -16.5486],
        [ -5.3734,  -6.8448,  -7.3414, -11.5521, -12.0292]], device='cuda:0')
[['\ufeffThe Project Gutenberation the pawns the ',
  "\ufeffThe Project Gutenberations the King's of",
  "\ufeffThe Project Gutenberations the King's th",
  "\ufeffThe Project Gutenberations the King's an",
  "\ufeffThe Project Gutenberations the King's to"],
 ['The Project Gutenberg.\n\n        ---------',
  'The Project Gutenberg.\n\n                 ',
  'The Project Gutenberg.\n\n          2. Kt-K',
  'The Project Gutenberg.\n\n          2. Kt-Q',
  'The Project Gutenberg.\n\n          2

In [12]:
idx, log_probabilities = net.predict(encoded[:5])

net.tensor2text(idx)

["\ufeffThe Project Gutenberations the King's of ",
 'The Project Gutenberg.\n\n        ----------',
 'he Project Gutenberg the pawn of the pawn ',
 "e Project Gutenberg Eove of the King's of ",
 ' Project Gutenberg EB\n        ------------']